#Pre-processing

##Import neccesary libraries

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import tensorflow as tf
from datasets import load_dataset
from transformers import BartTokenizer
import numpy as np
import re
import random
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup
import pandas as pd

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Load the data

In [ ]:
# load the cnn-dm dataset using dataset library
cnn_dm_dataset = load_dataset('cnn_dailymail', '3.0.0')

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
cnn_dm_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
pd.DataFrame(cnn_dm_dataset['train'][:100]).head()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


##Pre-processing dataset

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
def clean_entries(df):
    """Clean null entries in the DataFrame."""
    df.drop_duplicates(subset=['article'], inplace=True) # Drop duplicates
    df.dropna(axis=0,inplace=True) # Drop rows with missing values

    return df

In [ ]:
def clean_text(text):
    """Clean the text by removing extra whitespace, special characters, and stopwords."""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove special characters except punctuation
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(words).strip()

In [ ]:
dummy_df = pd.DataFrame(cnn_dm_dataset['train'][:100]).head()
dummy_df = clean_entries(dummy_df)
dummy_df['article'] = dummy_df['article'].apply(clean_text)
dummy_df['highlights'] = dummy_df['highlights'].apply(clean_text)
dummy_df

,article,highlights,id
0,"LONDON, England Reuters Harry Potter star Dani...",Harry Potter star Daniel Radcliffe gets 20M fo...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,"Editors note Behind Scenes series, CNN corresp...",Mentally ill inmates Miami housed forgotten fl...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota CNN Drivers Minneapolis...","NEW thought going die, driver says . Man says ...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON CNN Doctors removed five small poly...,Five small polyps found procedure none worriso...,24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,CNN National Football League indefinitely susp...,"NEW NFL chief, Atlanta Falcons owner critical ...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [ ]:
# Initialize the tokenizer (BART tokenizer in this case, adjust based on your model)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Set maximum lengths for input and output
MAX_INPUT_LENGTH = 1024
MAX_OUTPUT_LENGTH = 142  # Adjust based on your dataset/model

In [ ]:
def preprocess_function(examples):
    """Preprocess a batch of examples from the dataset."""
    try:
        articles = [clean_text(article) for article in examples['article']]
        summaries = [clean_text(summary) for summary in examples['highlights']]

        # Tokenize the input and target
        encoder_inputs = tokenizer(articles, max_length=MAX_INPUT_LENGTH, padding=True, truncation=True)
        decoder_inputs = tokenizer(summaries, max_length=MAX_OUTPUT_LENGTH, padding=True, truncation=True)

        # Shift decoder input tokens by one position (teacher forcing)
        decoder_input_ids = [[tokenizer.pad_token_id] + ids[:-1] for ids in decoder_inputs['input_ids']]

        return {
            'encoder_input_ids': encoder_inputs['input_ids'],
            'decoder_input_ids': decoder_input_ids,
            'labels': decoder_inputs['input_ids'],
            'encoder_attention_mask': encoder_inputs['attention_mask'],
            'decoder_attention_mask': decoder_inputs['attention_mask']
        }
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return {}

In [ ]:
# Apply preprocessing to the dataset with parallelism
preprocessed_dataset = cnn_dm_dataset.map(preprocess_function, batched=True, remove_columns=cnn_dm_dataset['train'].column_names, num_proc=8)

In [ ]:
preprocessed_dataset

In [ ]:
pd.DataFrame(preprocessed_dataset['train'][:100]).head()

##Saving the data

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Save train split
train_df = preprocessed_dataset['train'].to_pandas()
train_df.to_csv('/content/drive/MyDrive/SD_data/preprocessed_train.csv', index=False)

# Save validation split
val_df = preprocessed_dataset['validation'].to_pandas()
val_df.to_csv('/content/drive/MyDrive/SD_data/preprocessed_val.csv', index=False)

# Save test split
test_df = preprocessed_dataset['test'].to_pandas()
test_df.to_csv('/content/drive/MyDrive/SD_data/preprocessed_test.csv', index=False)

In [ ]:
####implement code here to load the pre_processed data

# Replace 'path/to/your/file.csv' with the actual path to your CSV file in Google Drive
file_path = '/content/drive/MyDrive/path/to/your/file.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/SD_data/preprocessed_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/SD_data/preprocessed_val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/SD_data/preprocessed_test.csv')

In [ ]:
train_df.head()

,encoder_input_ids,decoder_input_ids,labels,encoder_attention_mask,decoder_attention_mask
0,[ 0 574 4524 ... 1 1 1],[ 1 0 29345 10997 999 3028 7312 201...,[ 0 29345 10997 999 3028 7312 20152 15...,[1 1 1 ... 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
1,[ 0 44337 994 ... 1 1 1],[ 1 0 448 1342 2368 4812 8039 25...,[ 0 448 1342 2368 4812 8039 2561 157...,[1 1 1 ... 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
2,[ 0 24765 9009 ... 1 1 1],[ 1 0 5341 802 164 1597 6 13...,[ 0 5341 802 164 1597 6 1393 1...,[1 1 1 ... 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
3,[ 0 5762 3480 ... 1 1 1],[ 1 0 25496 650 11424 3275 303 70...,[ 0 25496 650 11424 3275 303 7089 41...,[1 1 1 ... 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...
4,[ 0 16256 496 ... 1 1 1],[ 1 0 5341 1485 834 6 3317 82...,[ 0 5341 1485 834 6 3317 8213 19...,[1 1 1 ... 0 0 0],[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...


##Continue

In [ ]:
# Convert to TensorFlow datasets
def to_tensorflow_dataset(dataset):
    """Convert the Hugging Face dataset to a TensorFlow Dataset."""
    # Convert each column to a list of strings
    # This will ensure TensorFlow datasets handles the string representation of numpy arrays correctly
    dataset_dict = {}
    for col in dataset.columns:
        dataset_dict[col] = dataset[col].astype(str).tolist()
    return tf.data.Dataset.from_tensor_slices(dataset_dict)

train_dataset = to_tensorflow_dataset(preprocessed_dataset['train'])
val_dataset = to_tensorflow_dataset(preprocessed_dataset['validation'])
test_dataset = to_tensorflow_dataset(preprocessed_dataset['test'])
# train_dataset = to_tensorflow_dataset(train_df)
# val_dataset = to_tensorflow_dataset(val_df)
# test_dataset = to_tensorflow_dataset(test_df)

In [ ]:
# Set up batching and prefetching for performance
BATCH_SIZE = 4  # Adjust based on your hardware

train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Print some statistics
print(f"Number of training examples: {len(train_df)}")
print(f"Number of validation examples: {len(val_df)}")
print(f"Number of test examples: {len(test_df)}")

# Example of accessing a batch
sample_batch = next(iter(train_dataset))
print("Sample batch shape:")
print(f"Encoder Input IDs: {sample_batch['encoder_input_ids'].shape}")
print(f"Decoder Input IDs: {sample_batch['decoder_input_ids'].shape}")
print(f"Labels: {sample_batch['labels'].shape}")

# Function to decode tokenized text
def decode_text(encoded_text):
    # Convert the Tensor to a NumPy array and then to a Python list
    decoded_text = tokenizer.decode(eval(encoded_text.numpy().decode()), skip_special_tokens=True)
    return decoded_text

# Print a sample article and its summary
sample_input = sample_batch['encoder_input_ids'][0]
sample_output = sample_batch['labels'][0]

print("\nSample Article:")
print(decode_text(sample_input))
print("\nSample Summary:")
print(decode_text(sample_output))

#Initial se2seq training

In [ ]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Concatenate, Input
from tensorflow.keras.models import Model

logger = tf.get_logger()

class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)

        # Create a trainable weight variable for this layer.
        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs

        def energy_step(inputs, states):
            encoder_full_seq = states[-1]
            W_a_dot_s = K.dot(encoder_full_seq, self.W_a)
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            e_i = K.softmax(e_i)
            return e_i, [e_i]

        def context_step(inputs, states):
            encoder_full_seq = states[-1]
            c_i = K.sum(encoder_full_seq * K.expand_dims(inputs, -1), axis=1)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)

        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e], constants=[encoder_out_seq]
        )
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c], constants=[encoder_out_seq]
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

# Clear previous Keras session
K.clear_session()

latent_dim = 500
max_len_text = 100  # Assuming some max_len for the input text
x_voc_size = 10000  # Assuming vocabulary size for the input
y_voc_size = 10000  # Assuming vocabulary size for the output

# Encoder
encoder_inputs = Input(shape=(max_len_text,))
enc_emb = Embedding(x_voc_size, latent_dim, trainable=True)(encoder_inputs)

# LSTM layers for encoder
encoder_lstm1 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm3(encoder_output2)

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, latent_dim, trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

# LSTM for decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention Layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concatenating attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

# Dense layer for output
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the Seq2Seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 100, 500)       │      5,000,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 100, 500),     │      2,002,000 │ embedding[0][0]        │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 100, 500),     │      2,002,000 │ lstm[0][0]             │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 500)      │      5,000,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 100, 500),     │      2,002,000 │ lstm_1[0][0]           │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 500),    │      2,002,000 │ embedding_1[0][0],     │
│                           │ (None, 500), (None,    │                │ lstm_2[0][1],          │
│                           │ 500)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_layer           │ [(None, None, 500),    │        500,500 │ lstm_2[0][0],          │
│ (AttentionLayer)          │ (None, None, 100)]     │                │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concat_layer              │ (None, None, 1000)     │              0 │ lstm_3[0][0],          │
│ (Concatenate)             │                        │                │ attention_layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, None, 10000)    │     10,010,000 │ concat_layer[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 28,518,500 (108.79 MB)

 Trainable params: 28,518,500 (108.79 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [ ]:
tr = '/content/drive/MyDrive/SD_data/preprocessed_train.csv'
val = '/content/drive/MyDrive/SD_data/preprocessed_val.csv'

In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=50,callbacks=[es],batch_size=512, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))


NameError: name 'x_tr' is not defined